In [18]:
import pandas as pd
import torch
import timm
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split


In [19]:

df = pd.read_csv("../tags_processed_stages/dafre_tags_symbolsremoved_minlen2_minapp2_profsremoved_filledempty.csv")
df.head()
print(df['tags_cat4'].nunique())

3381


In [27]:
class_counts = df['tags_cat4'].value_counts()
print(class_counts)

def contains_multiple_tags(label, separator=','):
    return separator in label
filtered_df = df[~df['tags_cat4'].apply(contains_multiple_tags, separator=',')]

class_counts = filtered_df['tags_cat4'].value_counts()
print(class_counts)

['hatsune miku']                                         10644
['hakurei reimu']                                         6691
['rumia']                                                 3803
['kochiya sanae']                                         3472
['cirno']                                                 3470
                                                         ...  
['alice margatroid', 'cardboard box gundam']                 1
['ayesha altugle', 'puni (atelier)']                         1
['akagi (azur lane)', 'shoukaku (azur lane)']                1
['maria traydor', 'rinoa heartilly']                         1
['hirasawa yui', 'kotobuki tsumugi', 'tainaka ritsu']        1
Name: tags_cat4, Length: 3381, dtype: int64
['hatsune miku']                           10644
['hakurei reimu']                           6691
['rumia']                                   3803
['kochiya sanae']                           3472
['cirno']                                   3470
                  

In [29]:
train_df, temp_test_df = train_test_split(filtered_df, test_size=0.2, stratify=filtered_df['tags_cat4'])
val_df, test_df = train_test_split(temp_test_df, test_size=0.5, stratify=temp_test_df['tags_cat4'])

print(train_df['tags_cat4'].nunique())
print(val_df['tags_cat4'].nunique())
print(test_df['tags_cat4'].nunique())

3258
3258
3258


In [ ]:
train_transforms = transforms.Compose({
    transforms.RandomResizedCrop((224, 224), scale=(0.8, 1)),
    transforms.TrivialAugmentWide(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
})
val_test_transforms = transforms.Compose({
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
})

In [ ]:
model = timm.create_model('beit_base_patch16_224_in22k', num_classes=3258, pretrained=True)
